## Generalized Disjunctive Programming

## Examples

https://towardsdatascience.com/schedule-optimisation-using-linear-programming-in-python-9b3e1bc241e1



A business needs to purchase new delivery vehicle for a regular set of routes totaling 1,500 km per day. The are several options as given in the following table.

| Vehicle | Range (km) | Fixed Cost (&euro;) | Operating Cost |
| :-----: | :---: | ---: |
| A | 200 | 40,000 |
| B | 150 | 30,000 |
| C | 400 | 50,000 |

To reduce training costs, the business will choose only one type of vehicle. Which should it be?

Customers

| Customer | Distance (RT) | Load |
| 1 | 120 | 300 |
| 2 | 80 | 120 |


In [58]:
import pandas as pd

vehicles = pd.DataFrame({
    "A": {"cost": 40000, "range": 200},
    "B": {"cost": 30000, "range": 150},
    "C": {"cost": 50000, "range": 250},
}).T

display(vehicles)

,cost,range
A,40000,200
B,30000,150
C,50000,250


This first attempt ceates a Disjunction.

In [78]:
import pyomo.environ as pyo
import pyomo.gdp as gdp

# model
m = pyo.ConcreteModel()

m.VEHICLES = pyo.Set(initialize=vehicles.index)
m.purchase = pyo.Var(m.VEHICLES, domain=pyo.NonNegativeIntegers, bounds=(0, 20))

@m.Objective(sense=pyo.minimize)
def cost(m):
    return sum(m.purchase[v]*vehicles.loc[v, "cost"] for v in m.VEHICLES)

@m.Disjunction()
def range_requirement(m):
    return [[m.purchase[v]*vehicles.loc[v, "range"] >= 1500] for v in m.VEHICLES]

# solve
pyo.TransformationFactory('gdp.hull').apply_to(m)
pyo.SolverFactory('glpk').solve(m)

# display
for v in m.VEHICLES:
    if m.purchase[v]() > 0:
        print(f"Purchase {m.purchase[v]()} vehicles at a cost {m.cost()}")

Purchase 6.0 vehicles at a cost 300000.0


Create separate Disjuncts, then combine the Disjuncts with a Disjunction. The reason to do this is that each Disjunct is a separate model block.

In [82]:
import pyomo.environ as pyo
import pyomo.gdp as gdp

# model
m = pyo.ConcreteModel()

m.VEHICLES = pyo.Set(initialize=vehicles.index)
m.purchase = pyo.Var(m.VEHICLES, domain=pyo.NonNegativeIntegers, bounds=(0, 200))

@m.Objective(sense=pyo.minimize)
def cost(m):
    return sum(m.purchase[v]*vehicles.loc[v, "cost"] for v in m.VEHICLES)

@m.Disjunct(m.VEHICLES)
def vehicle_range(b, v):
    m = b.model()
    b.charge
    b.range = pyo.Constraint(expr=m.purchase[v]*vehicles.loc[v, "range"] >= 1500)
    
@m.Disjunction()
def range(m):
    return [m.vehicle_range[v] for v in m.VEHICLES]
    
# solve
pyo.TransformationFactory('gdp.hull').apply_to(m)
pyo.SolverFactory('glpk').solve(m)

# display
for v in m.VEHICLES:
    if m.purchase[v]() > 0:
        print(f"Purchase {m.purchase[v]()} vehicles at a cost {m.cost()}")

Purchase 6.0 vehicles at a cost 300000.0


# Trivial Example

In [81]:
import pyomo.environ as pyo
import pyomo.gdp as gdp

m = pyo.ConcreteModel()

m.x = pyo.Var(domain=pyo.Binary)
m.y = pyo.Var(domain=pyo.Binary)
m.z = pyo.Var(domain=pyo.Binary)

@m.Objective(sense=pyo.maximize)
def obj(m):
    return m.x + m.y + m.z

@m.Disjunction(xor=True)
def conditions(m):
    return [[m.y ==0, m.z==0], 
            [m.x ==0]]
    
pyo.TransformationFactory('gdp.bigm').apply_to(m)
pyo.SolverFactory('glpk').solve(m)

print(m.x.value, m.y.value, m.z.value)
    

0.0 1.0 1.0


## Logic




In [7]:
import pyomo.environ as pyo
import pyomo.gdp as gdp

foods = {
    "fish": {"price": 12.80, "wine pair": ["white"]},
    "ham": {"price": 13.90, "wine pair": ["white", "red"]},
    "beef": {"price": 11.30, "wine pair:": ["red"]},
}

wines = {
    "cabernet": {"type": "red", "price": 8.30},
    "sauvignon blanc": {"type": "white", "price": 7.30},
    "pinot grigio": {"type": "white", "price": 4.50},
}

types = {wines[wine]["type"] for wine in wines.keys()}

print(foods)
print(wines)
print(types)

{'fish': {'price': 12.8, 'wine pair': ['white']}, 'ham': {'price': 13.9, 'wine pair': ['white', 'red']}, 'beef': {'price': 11.3, 'wine pair:': ['red']}}
{'cabernet': {'type': 'red', 'price': 8.3}, 'sauvignon blanc': {'type': 'white', 'price': 7.3}, 'pinot grigio': {'type': 'white', 'price': 4.5}}
{'red', 'white'}


In [17]:
m = pyo.ConcreteModel()

m.FOODS = pyo.Set(initialize=foods.keys())
m.WINES = pyo.Set(initialize=wines.keys())
m.TYPES = pyo.Set(initialize=list({wines[wine]["type"] for wine in wines.keys()}))
m.PAIRS = m.FOODS * m.WINES

#m.Pairings = pyo.Param(m.PAIRS, 
#                initialize = [1 if wines[wine]["type"] in foods[food]["wine pair"] else 0 for food, wine in m.PAIRS])

m.entre = pyo.Var(m.FOODS, domain=pyo.NonNegativeIntegers)
m.wine = pyo.Var(m.WINES, domain=pyo.NonNegativeIntegers)

m.meal_order = pyo.Constraint(expr=sum(m.entre[food] for food in m.FOODS) == 2)
m.wine_order = pyo.Constraint(expr=sum(m.wine[wine] for wine in m.WINES) >= 1)

m.cost = pyo.Objective(expr = sum(m.entre[food]*foods[food]["price"] for food in m.FOODS) +
                              sum(m.wine[wine]*wines[wine]["price"] for wine in m.WINES))

#@m.Disjunction()
#def pairings(m):
#    pass

pyo.SolverFactory('glpk').solve(m)

for food in m.FOODS:
    if m.entre[food].value > 0:
        print(food, m.entre[food].value)

for wine in m.WINES:
    if m.wine[wine].value > 0:
        print(wine, m.wine[wine].value)

beef 2.0
pinot grigio 1.0


Five people were eating apples, A finished before B, but behind C. D finished before E, but behind B. What was the finishing order?

In [3]:
import pyomo.environ as pyo

m = pyo.ConcreteModel()

m.people = pyo.Set(initialize=["A", "B", "C", "D", "E"])
m.finish = pyo.Var(m.people, domain=pyo.NonNegativeIntegers)
m.total = pyo.Var()

m.total_def = pyo.Constraint(m.people, rule=lambda m, p: m.finish[p] <= m.total)

m.obj = pyo.Objective(expr=m.total)

m.order = pyo.ConstraintList()
m.order.add(m.finish["A"] + 1 <= m.finish["B"])
m.order.add(m.finish["C"] + 1 <= m.finish["A"])
m.order.add(m.finish["D"] + 1 <= m.finish["E"])
m.order.add(m.finish["B"] + 1 <= m.finish["D"])

solver = pyo.SolverFactory('glpk')
%timeit solver.solve(m)

solver = pyo.SolverFactory('cbc')
%timeit solver.solve(m)

solver = pyo.SolverFactory('g')
%timeit solver.solve(m)

solver = pyo.SolverFactory('gurobi', solver_io="python")
%timeit solver.solve(m)

solver = pyo.SolverFactory('gurobi', executable="/usr/local/bin/gurobi.sh")
%timeit solver.solve(m)

soln = {p: m.finish[p]() for p in m.people}
for p in sorted(soln, key=soln.get):
    print(p)

17.8 ms ± 178 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
222 ms ± 8.91 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
1.69 ms ± 52.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
1.65 ms ± 40.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
77.3 ms ± 6.78 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
C
A
B
D
E


Five people were eating apples, A finished before B, but behind C. D finished before E, but behind B. What was the finishing order?

In [ ]:
import pyomo.environ as pyo




In [7]:
import pyomo.environ as pyo
import pyomo.gdp as gdp

m = pyo.ConcreteModel(name="Puzzle")

people = ["A", "B", "C", "D", "E"]
pairs = [(a, b) for a in people for b in people if a < b]

m.PERSONS = pyo.Set(initialize=people)
m.PAIRS = pyo.Set(initialize=pairs)

m.y = pyo.BooleanVar(m.PAIRS)

@m.LogicalConstraint(m.PAIRS, m.PAIRS)
def order(m, p, q, u, v):
    if q == u:
        return pyo.implies(pyo.land(m.y[p, q], m.y[q, v]), m.y[p, v])
    else:
        return pyo.Constraint.Skip

m.obj = pyo.Objective(expr = 0)

m.y[("A", "B")].fix(True)
m.y[("A", "C")].fix(False)
m.y[("D", "E")].fix(True)
m.y[("B", "D")].fix(False)

m.pprint()

pyo.TransformationFactory('gdp.hull').apply_to(m)
pyo.SolverFactory('glpk').solve(m)

for p, q in m.PAIRS:
    print(p, q, m.y[p, q]())

3 Set Declarations
    PAIRS : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     2 :    Any :   10 : {('A', 'B'), ('A', 'C'), ('A', 'D'), ('A', 'E'), ('B', 'C'), ('B', 'D'), ('B', 'E'), ('C', 'D'), ('C', 'E'), ('D', 'E')}
    PERSONS : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {'A', 'B', 'C', 'D', 'E'}
    order_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain      : Size : Members
        None :     4 : PAIRS*PAIRS :  100 : {('A', 'B', 'A', 'B'), ('A', 'B', 'A', 'C'), ('A', 'B', 'A', 'D'), ('A', 'B', 'A', 'E'), ('A', 'B', 'B', 'C'), ('A', 'B', 'B', 'D'), ('A', 'B', 'B', 'E'), ('A', 'B', 'C', 'D'), ('A', 'B', 'C', 'E'), ('A', 'B', 'D', 'E'), ('A', 'C', 'A', 'B'), ('A', 'C', 'A', 'C'), ('A', 'C', 'A', 'D'), ('A', 'C', 'A', 'E'), ('A', 'C', 'B', 'C'), ('A', 'C', 'B', 'D'), ('A', 'C', 'B', 'E'), ('A', 'C', 'C', 'D'), ('A', 'C', 'C', 

In [16]:
m.people

In [23]:
people = ["A", "B", "C", "D", "E"]

pairs = [(a, b) for a in people for b in people if a < b]